In [1]:
import networkx as nx
import numpy as np
import scipy.sparse as sp
import pickle
import torch
from torch_geometric.loader import DataLoader
from src.data.data_loader import load_input_data
from predictor import find_direction, find_feature
from predictor import load_models, predict_tableau
from src.CustomDataset import CustomDataset
from src.data.Data_gen_utils import shape_of_word
from src.data.data_loader import convert_networkx_to_adjacency_input, InputData
from src.data.generate_data import make_matrix_from_T

/home/chanhomin/anaconda3/envs/mpnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0"

In [3]:
MODELS = load_models()

  0 0.9958 8_4_256_all_with_all_row_connectedness_criterion_connected_UPTO_constant_222_conv


In [4]:
MODEL = MODELS[0]

In [5]:
with open(MODEL, 'rb') as f:
    model, acc, loss = pickle.load(f)
    model.to(device)

In [6]:
DIR_PATH = "./Data/n=5_all_with_inductive_connectedness_criterion_connected_UPTO/"

print("Loading input data...")
full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

Loading input data...
Generating data from the directory ./Data/n=5_all_with_inductive_connectedness_criterion_connected_UPTO/


In [7]:
batch_size = 1

test_dataset = CustomDataset(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
with torch.no_grad():
    for batch in test_loader:
        batch.to(device)
        outputs = model(batch)
        print(outputs)
        break

/home/chanhomin/Ptab/Ptab/src/CustomDataset.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)


tensor([[0.9936, 0.0064]], device='cuda:0')


for data in test_dataset:
    data1= data
    break

In [26]:
P = [2, 3, 4, 5, 6, 7, 7]
word = [3, 2, 1, 4, 5, 6, 7]
predict_tableau(P, word, MODEL)

TypeError: 'tuple' object is not callable

In [11]:
features = find_feature(MODEL)

In [12]:
features

{'constant': (True, <function src.data.feature_functions.constant_feature(D)>)}

In [15]:
shape = shape_of_word(P, word)
print(shape)

direction1, direction2, direction3 = find_direction(MODEL)
# direction1, direction2, direction3 = Direction.FORWARD, Direction.FORWARD, Direction.FORWARD
features = find_feature(MODEL)

T = make_matrix_from_T(P, word, direction=(direction1, direction2, direction3))
graph = nx.from_scipy_sparse_array(T, create_using=nx.DiGraph)

feat_dict = dict()
for key, value in features.items():
    feat_dict[key] = value[1](graph)

feature = np.zeros((len(graph), len(feat_dict)))

for n, node in enumerate(graph.nodes):
    for i, (key, value) in enumerate(feat_dict.items()):
        feature[n, i] = value[node]

features = [feature]
ys = np.array([1])
adjacencies = [convert_networkx_to_adjacency_input(graph)]

rows = [np.array(sp.coo_matrix(a).row, dtype=np.int16) for a in adjacencies]
cols = [np.array(sp.coo_matrix(a).col, dtype=np.int16) for a in adjacencies]
edge_types = [np.array(sp.coo_matrix(a).data, dtype=np.int16) for a in adjacencies]
graphs_sizes = [len(graph.nodes)]
print(len(graph.nodes))
# T_inputdata = InputData(features=features, labels=ys, rows=rows, columns=cols, edge_types=edge_types)
T_inputdata = InputData(features=features, labels=ys, rows=rows, columns=cols, edge_types=edge_types, graph_sizes=graphs_sizes)
T_dataset = CustomDataset(T_inputdata)
T_loader = DataLoader(T_dataset, batch_size=1, shuffle=True)


[7]
7


In [16]:
rows

[array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6,
        6, 6, 6, 6, 6], dtype=int16)]

In [17]:
edge_types

[array([1, 4, 3, 3, 3, 3, 3, 4, 1, 4, 3, 3, 3, 3, 3, 4, 1, 4, 3, 3, 3, 3,
        3, 4, 1, 4, 3, 3, 3, 3, 3, 4, 1, 4, 3, 3, 3, 3, 3, 4, 1, 4, 3, 3,
        3, 3, 3, 4, 1], dtype=int16)]

In [18]:
graphs_sizes

[7]

In [19]:
T_loader

In [20]:
import pickle

In [21]:
with open(MODEL, 'rb') as f:
    model, acc, loss = pickle.load(f)
    model.to(device)

In [22]:
for batch in T_loader:
    batch.to(device)
    print(batch.batch)
    print(batch.x)
    print(batch.edge_index)
    print(batch.edge_types)
    predicted = model(batch)
    print(predicted)
    print("---------")

tensor([0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
         3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
         6],
        [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
         3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
         6]], device='cuda:0')
tensor([[1.],
        [4.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [4.],
        [1.],
        [4.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [4.],
        [1.],
        [4.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [4.],
        [1.],
        [4.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [4.],
        [1.],
       

In [23]:
batch.x

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [24]:
tt.x

NameError: name 'tt' is not defined

In [ ]:
tt.edge_types

In [25]:
for tt in test_loader:
    tt.to(device)
    print(tt.batch)
    print(tt.x)
    print(tt.edge_index)
    print(tt.edge_types)
    
    predicted = model(batch)
    print(predicted)
    print("---------")
    break

tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4],
        [0, 1, 2, 3, 4, 1, 4, 1, 2, 3, 4, 1, 3, 4, 4]], device='cuda:0')
tensor([[1.],
        [4.],
        [4.],
        [3.],
        [3.],
        [1.],
        [3.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [1.]], device='cuda:0')
tensor([[1.8233e-04, 9.9982e-01]], device='cuda:0', grad_fn=<SliceBackward0>)
---------
